In [1]:
include("JuliRay.jl")
Base.@irrational ° 0.0174532925199432957692369076848861271344 (big(pi)/big(180))

In [2]:
ε=10^(-3)

function f(x)
    ifelse(x>0,exp(-1/x),0)
end
function g(x)
    f(2x/√3)/(f(2x/√3)+f(2/√3-2x/√3))
end
function Smooth(a,b,x)
    g((x-a)/(b-a))
end

VERTEX=Int
EDGE=Array{VERTEX,1}
FACE=Array{EDGE,1}
CELL=Array{FACE,1}

function ℝ³⭢S³(p::Array{T,1}) where T<:Real
    if(length(p)≠3)
        error("No Point on ℝ³")
    end
    return [2p[1]/(1+p[1]^2+p[2]^2+p[3]^2),2p[2]/(1+p[1]^2+p[2]^2+p[3]^2),2p[3]/(1+p[1]^2+p[2]^2+p[3]^2),(-1+p[1]^2+p[2]^2+p[3]^2)/(1+p[1]^2+p[2]^2+p[3]^2)]
end

function ℝ³⭢S³(p::Array{T,1},θ::Real) where T<:Real
    if(length(p)≠3)
        error("No Point on ℝ³")
    elseif(θ<ε)
        return [p[1:3]...,0.0]
    else
        c=[0,0,0,cot(θ)]
        r=1/sin(θ)
        P=p*cot((π/2-θ)/2+π/4)
        return ℝ³⭢S³(P)*r+c
    end
end

function S³⭢ℝ³(q::Array{T,1}) where T<:Real
    if(length(q)≠4)
        error("No Point on S³")
    elseif(!(norm(q)≈1.0))
        print(norm(q))
        error("No Point on unit S³, $(norm(q))")
    end
    return [q[1]/(1-q[4]),q[2]/(1-q[4]),q[3]/(1-q[4])]
end

function S³⭢ℝ³(q::Array{T,1},θ::Real) where T<:Real
    if(length(q)≠4)
        error("No Point on S³")
    elseif(θ<ε)
        return q[1:3]
    else
        c=[0,0,0,cot(θ)]
        r=1/sin(θ)
        return S³⭢ℝ³((q-c)/r)*tan((π/2-θ)/2+π/4)
    end
end

function ℝ⁴⭢S³(p::Array{T,1},θ::Real) where T<:Real
    if(θ<ε)
        return [p[1:3]...,0.0]
    else
        R=1/sin(θ)
        O=[0,0,0,cot(θ)]
        return O+R*normalize(p-O)
    end
end

function Mirror(q::RealVector,p₁::RealVector,p₂::RealVector,p₃::RealVector)
    𝒏=NormalVector(p₁,p₂,p₃)
    return q-2*dot(𝒏,q-p₁)*𝒏
end

function NormalVector(p₁::RealVector,p₂::RealVector,p₃::RealVector,p₄::RealVector)
    A=hcat(p₁-p₄,p₂-p₄,p₃-p₄)
    return normalize([(-1)^i*det(A[deleteat!(collect(1:4),i),:]) for i ∈ 1:4])
end

function Mirror(q::RealVector,p₁::RealVector,p₂::RealVector,p₃::RealVector,p₄::RealVector)
    𝒏=NormalVector(p₁,p₂,p₃,p₄)
    return q-2*dot(𝒏,q-p₁)*𝒏
end

function vertices(face::FACE)
    f=copy(face)
    v=union(vcat(f...))
    w=[v[1]]
    for j ∈ 1:(length(v)-1)
        i=findfirst(e->w[end] ∈ e, f)
        push!(w,filter(v->v≠w[end],f[i])[1])
        deleteat!(f,i)
    end
    return w
end

function PickFace(cell::CELL, v::RealVector,POINTS³)
    if(norm(v)≈0)
        error("vector v must be non-zero")
    end
    faces=vertices.(cell)
    fpts=(i->POINTS³[i]).(faces)
    return cell[findmax([dot(v,+(pts...)) for pts ∈ fpts])[2]]
end

function NewCell(cell::CELL,v::RealVector,θ,POINTS³, POINTS⁴)
    n=length(POINTS³)
    c=copy(cell)
    face=PickFace(cell,v,POINTS³)
    IND_face=vertices(face)
    IND_cell=Int[]
    for f ∈ c for e ∈ f for v ∈ e push!(IND_cell,v) end end end
    IND_cell=union(IND_cell)
    IND_cell2=[
        if(i ∈ IND_face)
            i
        else
            push!(POINTS³,Mirror(POINTS³[i],POINTS³[IND_face[1]],POINTS³[IND_face[2]],POINTS³[IND_face[3]]));
            if(θ<ε)
                push!(POINTS⁴,[POINTS³[end]...,0]);
            else
                O=[0,0,0,cot(θ)]
                push!(POINTS⁴,Mirror(POINTS⁴[i],O,POINTS⁴[IND_face[1]],POINTS⁴[IND_face[2]],POINTS⁴[IND_face[3]]));
            end
            n=n+1
        end
        for i ∈ IND_cell]
    return [[[IND_cell2[findfirst(w->w==v,IND_cell)] for v ∈ e] for e ∈ f] for f ∈ c]
end

function SphericalSphere(v,r::Real,θ::Real) where T<:RealVector
    V=S³⭢ℝ³(v,θ)
    return Sphere(V,r)
end

function SphericalCylinder(v₁,v₂,r::Real,θ::Real) where T<:RealVector
    w₁=ℝ⁴⭢S³((v₁+v₂)/2,θ)
    V₁=S³⭢ℝ³(v₁,θ)
    V₂=S³⭢ℝ³(v₂,θ)
    W₁=S³⭢ℝ³(w₁,θ)
    return Arc(V₁,W₁,V₂,r)
end

function SphericalPolygon(v::Array{T,1},θ::Real) where T<:RealVector
    n=length(v)
    u=ℝ⁴⭢S³(+(v...)/n,θ)
    v₁=v[1]
    v₂=v[2]
    v₃=v[3]
    V=(q->S³⭢ℝ³(q,θ)).(v)
    U=S³⭢ℝ³(u,θ)
    w=[ℝ⁴⭢S³((v[i]+v[mod(i,length(v))+1])/2,θ) for i ∈ 1:length(v)]
    W=(q->S³⭢ℝ³(q,θ)).(w)
    if(θ<ε)
        return Polygon(V)
    elseif(rank(hcat(V...),atol=1.0e-12)==2)
        m=4
        vw=copy(v)
        for _ ∈ 1:m
            l=length(vw)
            vw=[ℝ⁴⭢S³((vw[(i+1)÷2]+vw[mod(i÷2,l)+1])/2,θ) for i ∈ 1:2l]
        end
        VW=(q->S³⭢ℝ³(q,θ)).(vw)
        return csgUnion(Polygon(VW))
    else
        O=Circumcenter(U,V[1],V[2],V[3])
        sphere=Sphere(O,norm(U-O))
        N=NormalVector(V[1],W[1],V[2]);
        direction=sign(dot(N,O-U))

        cylinders=csgIntersection([
                (V₁=V[i];
                V₂=V[mod(i,n)+1];
                V₃=V[mod(i+1,n)+1];
                C=Circumcenter(V₁,W[i],V₂);
                N=NormalVector(V₁,W[i],V₂);
                direction=sign(dot(N,U-W[i]));
                cylinder=Cylinder(C,C+2*direction*norm(U-O)*N,norm(U-O)))
                for i ∈ 1:n
                ])
        return csgClip(sphere,cylinders)
    end
end

function Cells2Object(cells::Array{CELL,1},θ,POINTS⁴;rᵥ=0.05,rₑ=0.025,color=RGB(1,1,1))
    cs=copy(cells)
    fs=DeleteDuplicates(vcat(cs...))
    es=DeleteDuplicates(vcat(fs...))
    vs=DeleteDuplicates(vcat(es...))
    V=rgbColor(csgUnion([SphericalSphere(POINTS⁴[v],rᵥ,θ) for v ∈ vs]),RGB(0.1,0.1,0.1))
    E=rgbColor(csgUnion([SphericalCylinder(POINTS⁴[e[1]],POINTS⁴[e[2]],rₑ,θ) for e ∈ es]),RGB(0.2,0.2,0.2))
    F=rgbftColor(csgUnion([SphericalPolygon([POINTS⁴[i] for i ∈ vertices(f)],θ) for f ∈ fs]),color,FT(0.1,0.3))
    return csgUnion(V,E,F)
end

Cells2Object (generic function with 1 method)

In [3]:
function Tesseract(θ)
    h=1/2
    r=√(1-h^2)

    𝒑₁=[1,1,1]
    𝒑₂=[1,1,-1]
    𝒑₃=[1,-1,1]
    𝒑₄=[1,-1,-1]
    𝒑₅=[-1,1,1]
    𝒑₆=[-1,1,-1]
    𝒑₇=[-1,-1,1]
    𝒑₈=[-1,-1,-1]
    POINTS=[𝒑₁,𝒑₂,𝒑₃,𝒑₄,𝒑₅,𝒑₆,𝒑₇,𝒑₈]
    POINTS=r*normalize.(POINTS)
    R=1/sin(θ)
    O=[0,0,0,cot(θ)]
    N=O+[0,0,0,R]
    H=O-[0,0,0,√(R^2-r^2)]
    POINTS³=copy(POINTS)
    POINTS⁴=[[𝒑...,0]+H for 𝒑 ∈ POINTS]

    v₁=1
    v₂=2
    v₃=3
    v₄=4
    v₅=5
    v₆=6
    v₇=7
    v₈=8
    e₁=[v₁,v₂]
    e₂=[v₁,v₃]
    e₃=[v₁,v₅]
    e₄=[v₂,v₄]
    e₅=[v₂,v₆]
    e₆=[v₃,v₄]
    e₇=[v₃,v₇]
    e₈=[v₄,v₈]
    e₉=[v₅,v₆]
    e₁₀=[v₅,v₇]
    e₁₁=[v₆,v₈]
    e₁₂=[v₇,v₈]
    f₁=[e₁,e₂,e₄,e₆]
    f₂=[e₁,e₃,e₅,e₉]
    f₃=[e₂,e₃,e₇,e₁₀]
    f₄=[e₄,e₅,e₈,e₁₁]
    f₅=[e₆,e₇,e₈,e₁₂]
    f₆=[e₉,e₁₀,e₁₁,e₁₂]
    c₁=[f₁,f₂,f₃,f₄,f₅,f₆]
    c₂=NewCell(c₁,[-1,0,0],θ,POINTS³, POINTS⁴)
    c₃=NewCell(c₂,[-1,0,0],θ,POINTS³, POINTS⁴)
    c₄=NewCell(c₁,[1,0,0],θ,POINTS³, POINTS⁴)
    c₅=NewCell(c₁,[0,0,-1],θ,POINTS³, POINTS⁴)
    c₆=NewCell(c₅,[0,-1,0],θ,POINTS³, POINTS⁴)
    c₇=NewCell(c₆,[0,-1,0],θ,POINTS³, POINTS⁴)
    c₈=NewCell(c₅,[0,1,0],θ,POINTS³, POINTS⁴)
    
    cells=[c₁,c₂,c₃,c₄,c₅,c₆,c₇,c₈]
    return cells, POINTS³, POINTS⁴
end

Tesseract (generic function with 1 method)

In [4]:
# 14 min
M=60
for i ∈ 0:2M
    θ=π/2*(Smooth(0,1,i/M)-Smooth(1,2,i/M))
    cells, POINTS³, POINTS⁴=Tesseract(θ)
    render(Cells2Object(cells,θ,POINTS⁴,color=RGB(0,1,1)),camera=LngLatCamera(lng=180°+360°*i/M,lat=25°,pers=0.2,zoom=0.15,width=800,height=600),name="Tesseract_",index=i+1)
end

povray: cannot open the user configuration file /home/horikawa/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Dilger      Alexande

POV-Ray finished

povray: cannot open the user configuration file /home/horikawa/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Di

POV-Ray finished

povray: cannot open the user configuration file /home/horikawa/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Di

POV-Ray finished

povray: cannot open the user configuration file /home/horikawa/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Di

POV-Ray finished

povray: cannot open the user configuration file /home/horikawa/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Di

...

Rendered 480000 of 480000 pixels (100%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 800 x 600
----------------------------------------------------------------------------
Pixels:           510200   Samples:          150840   Smpls/Pxl: 0.30
Rays:             996035   Saved:                 0   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage
----------------------------------------------------------------------------
Box                           97088582        38452505     39.61
CSG Intersection              97088582         9627700      9.92
Sphere                        72470679        66007012     91.08
Torus                         48544291          498075      1.03
Torus Bound                   48544291          781346      1.61
Clipping Object               82160905         1263925      1.54
Bounding Bo

Parser Options
  Input file: Tesseract__000016.pov
  Remove bounds........On 
  Split unions.........Off
  Library paths:
  Clock value:    0.000  (Animation off)
Image Output Options
  Image resolution.....800 by 600 (rows 1 to 600, columns 1 to 800).
  Output file..........Tesseract__000016.png, 24 bpp PNG
  Dithering............Off
  Graphic display......Off
  Mosaic preview.......Off
  Continued trace......Off
Information Output Options
  All Streams to console..........On 
  Debug Stream to console.........On 
  Fatal Stream to console.........On 
  Render Stream to console........On 
  Statistics Stream to console....On 
  Warning Stream to console.......On 
==== [Parsing...] ==========================================================
----------------------------------------------------------------------------
Parser Statistics
----------------------------------------------------------------------------
Finite Objects:          145
Infinite Objects:          0
Light Sources:      

POV-Ray finished

povray: cannot open the user configuration file /home/horikawa/.povray/3.7/povray.conf: No such file or directory
Persistence of Vision(tm) Ray Tracer Version 3.7.0.8.unofficial (g++ 8.2.1 @
 x86_64-pc-linux-gnu)
This is an unofficial version compiled by:
 Arch Linux
 The POV-Ray Team is not responsible for supporting this version.

POV-Ray is based on DKBTrace 2.12 by David K. Buck & Aaron A. Collins
Copyright 1991-2013 Persistence of Vision Raytracer Pty. Ltd.

Primary POV-Ray 3.7 Architects/Developers: (Alphabetically)
  Chris Cason         Thorsten Froehlich  Christoph Lipka   

With Assistance From: (Alphabetically)
  Nicolas Calimet     Jerome Grimbert     James Holsenback    Christoph Hormann 
  Nathan Kopp         Juha Nieminen     

Past Contributors: (Alphabetically)
  Steve Anger         Eric Barish         Dieter Bayer        David K. Buck     
  Nicolas Calimet     Chris Cason         Aaron A. Collins    Chris Dailey      
  Steve Demlow        Andreas Di

Rendered 480000 of 480000 pixels (100%)
----------------------------------------------------------------------------
Render Statistics
Image Resolution 800 x 600
----------------------------------------------------------------------------
Pixels:           510200   Samples:          157869   Smpls/Pxl: 0.31
Rays:            1034630   Saved:                 0   Max Level: 1/5
----------------------------------------------------------------------------
Ray->Shape Intersection          Tests       Succeeded  Percentage
----------------------------------------------------------------------------
Box                          106018828        40170098     37.89
CSG Intersection             106018828         9478769      8.94
Sphere                        73106785        65695617     89.86
Torus                         53009414          546181      1.03
Torus Bound                   53009414          814711      1.54
Clipping Object               84986398         1401289      1.65
Bounding Bo

Parser Options
  Input file: Tesseract__000021.pov
  Remove bounds........On 
  Split unions.........Off
  Library paths:
  Clock value:    0.000  (Animation off)
Image Output Options
  Image resolution.....800 by 600 (rows 1 to 600, columns 1 to 800).
  Output file..........Tesseract__000021.png, 24 bpp PNG
  Dithering............Off
  Graphic display......Off
  Mosaic preview.......Off
  Continued trace......Off
Information Output Options
  All Streams to console..........On 
  Debug Stream to console.........On 
  Fatal Stream to console.........On 
  Render Stream to console........On 
  Statistics Stream to console....On 
  Warning Stream to console.......On 
==== [Parsing...] ==========================================================
----------------------------------------------------------------------------
Parser Statistics
----------------------------------------------------------------------------
Finite Objects:          145
Infinite Objects:          0
Light Sources:      

In [5]:
function C₁₆(θ)
    h=1/2
    r=√(1-h^2)

    𝒑₁=[1,1,1]
    𝒑₂=[1,-1,-1]
    𝒑₃=[-1,1,-1]
    𝒑₄=[-1,-1,1]
    POINTS=[𝒑₁,𝒑₂,𝒑₃,𝒑₄]
    POINTS=r*normalize.(POINTS)
    R=1/sin(θ)
    O=[0,0,0,cot(θ)]
    N=O+[0,0,0,R]
    H=O-[0,0,0,√(R^2-r^2)]
    POINTS³=copy(POINTS)
    POINTS⁴=[[𝒑...,0]+H for 𝒑 ∈ POINTS]

    v₁=1
    v₂=2
    v₃=3
    v₄=4
    e₁=[v₁,v₂]
    e₂=[v₁,v₃]
    e₃=[v₁,v₅]
    e₄=[v₂,v₄]
    e₅=[v₂,v₆]
    e₆=[v₃,v₄]
    f₁=[e₁,e₂,e₃]
    f₂=[e₁,e₃,e₅,e₉]
    f₃=[e₂,e₃,e₇,e₁₀]
    f₄=[e₄,e₅,e₈,e₁₁]
    c₁=[f₁,f₂,f₃,f₄]
    c₂=NewCell(c₁,[-1,0,0],θ,POINTS³, POINTS⁴)
    
    cells=[c₁,c₂,c₃,c₄,c₅,c₆,c₇,c₈]
    return cells, POINTS³, POINTS⁴
end

LoadError: syntax: invalid character "𝟏" near column 11